## load library

In [1]:
from helper import *
import sys

## load config

In [2]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)

# display(minio_config.__len__())
# display(minio_config)

## set variables

In [3]:
## set api_config variables
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
topic = "service_opd_screening"
api_config["headers"] = headers
api_config["topic"] = topic

try:
  r = get_process(**api_config)
except Exception as ex:
  print(ex)
  sys.exit(0)

## set config to processing 
hospcode = r["hospcode"]
process_id = r["process_id"]
api_config["process_id"] = process_id
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/hisgw/{}/{}/{}".format(hospcode, topic,process_id)
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

# display(r.__len__())
# display(r)


## load person

In [4]:
person = read_csv(minio_config, path=source_path, filename="person.csv.gz")

# display(person.__len__())
# display(person)

## merge localcode to person

In [5]:
person_merge_local = dict(
    title=["title_code", "code"],
    gender=["gender", "code"],
    rh_blood_group=["rh_blood_group", "code"],
    blood_group=["blood_group", "code"],
    nationality=["nationality_code", "code"],
    marital_status=["marital_status", "code"],
)
person = merge_codes(minio_config, person, local_code_path,
                     prefix_type="local", merge_config=person_merge_local)

# display(person.__len__())
# display(person)

## merge stdcode to person

In [6]:
person_merge_std = dict(
    titles=["title_localcode", "localcode"],
    genders=["gender_localcode", "localcode"],
    rh_blood_groups=["rh_blood_group_localcode", "localcode"],
    blood_groups=["blood_group_localcode", "localcode"],
    nationalities=["nationality_localcode", "localcode"],
    marital_statuses=["marital_status_localcode", "localcode"],
)
person = merge_codes(minio_config, person, std_code_path,
                     prefix_type="std", merge_config=person_merge_std)
# person = person.drop_duplicates(subset=["hn"])

# display(person.__len__())
# display(person)

## rename columns person

In [7]:
cols_rename = dict(
    group="groups",
    nationality="nationalities",
    status="statuses",
    gender="genders",
    title="titles"
)
person = rename_columns(person, cols_rename)

# display(person.__len__())
# display(person)

## load service_opd

In [8]:
service_opd = read_csv(
    minio_config, path=source_path, filename="service_opd.csv.gz")

# try:
#     service_opd = read_csv(
#         minio_config, path=source_path, filename="service_opd.csv.gz")
# except Exception as ex:
#     print(ex)

# display(service_opd.__len__())
# display(service_opd)

## merge local service_opd

In [9]:
service_opd_merge_local = dict(
    main_department=["main_department", "code"],
    sub_department=["sub_department", "code"],
)
service_opd = merge_codes(minio_config, service_opd, local_code_path,
                     prefix_type="local", merge_config=service_opd_merge_local)

# display(service_opd.__len__())
# display(service_opd)

## merge std service_opd

In [10]:
service_opd_merge_std = dict(
    main_departments_opd=["main_department", "localcode"],
    # sub_department=["sub_department", "code"],
)
service_opd = merge_codes(minio_config, service_opd, std_code_path,
                          prefix_type="std", merge_config=service_opd_merge_std)

# display(service_opd.__len__())
# display(service_opd)

## rename service_opd columns

In [11]:
cols_rename = dict(
    department = "departments_opd",
)
service_opd = rename_columns(service_opd, cols_rename)

# display(service_opd.__len__())
# display(service_opd)

## load service_opd_screening
- ไม่มี localcode && stdcode

In [12]:
service_opd_screening = read_csv(minio_config, path=source_path, filename="service_opd_screening.csv.gz")

# display(service_opd_screening.__len__())
# display(service_opd_screening)

## merge localcode to service_opd_screening

In [13]:
## service_opd_screening doesn,t have localcode

## merge stdcode to service_opd_screening

In [14]:
# service_opd_screening doesn,t have stdcode

## merge service_opd_screeing to service_opd

In [15]:
service_opd_screening = pd.merge(service_opd_screening, service_opd, how="left", left_on="vn", right_on="vn", suffixes=("", "_right"))
service_opd_screening = service_opd_screening[[a for a in service_opd_screening.columns.to_list() if a.endswith("_right")==False]]

# display(service_opd_screening.__len__())
display(service_opd_screening)

,vn,hn,bpd,bps,weight,chief_complain,hearth_rate,pulse_rate,temperature,respiratory_rate,height,waist,an,visit_date,visit_time,main_department,sub_department,pttype,diagnosis_text,main_department_localcode,main_department_localcode,main_department_localname,sub_department_localcode,sub_department_localname,main_department_localcode,main_department_localcode,main_department_stdcode,main_department_stdname_th,main_department_stdname_en
0,600712082919,000033167,0,0,0,ญาติรับยาแทน case poag ไม่ขาดยา ขอรับยาต่อ อา...,<NA>,0,0,0,155,107,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,610316070300,000082057,60,100,50,PCEC 0.1 ML * 2 จุด ID เข็มที่3/ไกลกว่าRN,<NA>,82,36.5,16,150,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,600905010716,000373909,0,0,15,มาามนัด case EKG LE อาการ ดีขึ้น ไม่คัน ยุบบวม,<NA>,100,36.9,18,110,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,601108010926,000017101,76,138,57,ข้อนิ้วมือติดแข็ง,<NA>,80,0,18,154,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,610317135250,000226044,66,117,47,ถูกทำร้ารร่างกายโดยใช้กำปั้นต่อยที่ใบหน้ามีแผล...,<NA>,82,37,20,155,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,650808181839,000529071,0,0,13,ไข้ ตุ่มที่เท้าและปาก 3 วันก่อนมารพ.,<NA>,120,36.7,24,84,0,<NA>,2022-08-07T17:00:00.000Z,18:18:39,อายุรกรรม,จุดคัดกรองหน้าห้องตรวจ (อายุกรรม) เบอร์ 9,71,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
996,650808133206,000527541,88,136,64,case ARV มาตามนัด 4 เดือนรับยาต่อเนื่อง อาการ...,<NA>,87,37,20,158,83,<NA>,2022-08-07T17:00:00.000Z,13:32:06,อายุรกรรม,คลินิก ARV,01,"HIV POSITIVE,HIV POSITIVE",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
997,650808081012,000516659,50,80,13,ไข้ มีตุ่มตามก้น มือ2ข้าง ตุ่มแดงฝ่ามือ มาตามน...,<NA>,80,36.5,20,90,0,<NA>,2022-08-07T17:00:00.000Z,08:10:12,กุมารเวชกรรม,จุดคัดกรองหน้าห้องตรวจ (กุมารเวชกรรม) เบอร์ 13,71,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
998,650808101904,000397145,84,122,57,ขอทำกายภาพกายบำบัด เรื่อง หลัง,<NA>,70,36,20,155,0,<NA>,2022-08-07T17:00:00.000Z,10:19:04,ศัลยกรรมกระดูก,จุดคัดกรองหน้าห้องตรวจ (ศัลยกรรมกระดูก) เบอร์ 1,51,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## merge person to service_opd_screening

In [16]:
service_opd_screening = pd.merge(
    service_opd_screening, person, how="left", left_on="hn", right_on="hn", suffixes=("", "_right"))
service_opd_screening = service_opd_screening[[
    a for a in service_opd_screening.columns.to_list() if a.endswith("_right") == False]]

# display(service_opd_screening.__len__())
display(service_opd_screening)

,vn,hn,bpd,bps,weight,chief_complain,hearth_rate,pulse_rate,temperature,respiratory_rate,height,waist,an,visit_date,visit_time,main_department,sub_department,pttype,diagnosis_text,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localname,sub_department_localcode,sub_department_localname,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_localcode,main_department_stdcode,main_department_stdname_th,main_department_stdname_en,cid,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status,title_localcode,title_localcode,title_localname,gender_localcode,gender_localcode,gender_localname,rh_blood_group_localcode,rh_blood_group_localcode,rh_blood_group_localname,blood_group_localcode,blood_group_localcode,blood_group_localname,nationality_localcode,nationality_localcode,nationality_localname,marital_status_localcode,marital_status_localcode,marital_status_localname,title_localcode,title_localcode,title_stdcode,title_stdname_th,title_stdname_en,gender_localcode,gender_localcode,gender_stdcode,gender_stdname_th,gender_stdname_en,rh_blood_group_localcode,rh_blood_group_localcode,rh_blood_group_stdcode,rh_blood_group_stdname_th,rh_blood_group_stdname_en,blood_group_localcode,blood_group_localcode,blood_group_stdcode,blood_group_stdname_th,blood_group_stdname_en,nationality_localcode,nationality_localcode,nationality_stdcode,nationality_stdname_th,nationality_stdname_en,marital_status_localcode,marital_status_localcode,marital_status_stdcode,marital_status_stdname_th,marital_status_stdname_en
0,600712082919,000033167,0,0,0,ญาติรับยาแทน case poag ไม่ขาดยา ขอรับยาต่อ อา...,<NA>,0,0,0,155,107,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310401448357,นาย,ใจ,<NA>,สวัสดิ์พูน,1926-06-29T17:00:00.000Z,1,<NA>,ไม่ทราบหมู่เลือด,99,3,นาย,<NA>,นาย,1,1,ชาย,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,ไทย,<NA>,<NA>,<NA>,นาย,<NA>,<NA>,<NA>,<NA>,1,1,1,ชาย,Male,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,99,ไทย,THAILAND,<NA>,<NA>,9,อื่นๆ,Other
1,610316070300,000082057,60,100,50,PCEC 0.1 ML * 2 จุด ID เข็มที่3/ไกลกว่าRN,<NA>,82,36.5,16,150,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310400125406,นาง,เฉลียว,<NA>,สวนสวรรค์,1950-12-31T17:00:00.000Z,2,<NA>,ไม่ทราบหมู่เลือด,99,2,นาง,<NA>,นาง,2,2,หญิง,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,ไทย,<NA>,<NA>,<NA>,นาง,<NA>,<NA>,<NA>,<NA>,2,2,2,หญิง,Female,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,99,ไทย,THAILAND,<NA>,<NA>,9,อื่นๆ,Other
2,600905010716,000373909,0,0,15,มาามนัด case EKG LE อาการ ดีขึ้น ไม่คัน ยุบบวม,<NA>,100,36.9,18,110,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1240600312930,ด.ญ.,สุจิตา,<NA>,สมัญญา,2011-08-24T17:00:00.000Z,2,<NA>,ไม่ทราบหมู่เลือด,99,1,ด.ญ.,<NA>,ด.ญ.,2,2,หญิง,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,ไทย,<NA>,<NA>,<NA>,ด.ญ.,<NA>,<NA>,<NA>,<NA>,2,2,2,หญิง,Female,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,99,ไทย,THAILAND,<NA>,<NA>,9,อื่นๆ,Other
3,601108010926,000017101,76,138,57,ข้อนิ้วมือติดแข็ง,<NA>,80,0,18,154,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310401337300,นาง,พเนาว์,<NA>,โตสกุล,1965-09-22T17:00:00.000Z,2,<NA>,ไม่ทราบหมู่เลือด,99,2,นาง,<NA>,นาง,2,2,หญิง,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,ไทย,<NA>,<NA>,<NA>,นาง,<NA>,<NA>,<NA>,<NA>,2,2,2,หญิง,Female,<NA>,<NA>,<NA>,<NA>,<NA>,<N

## set export status variable
- ใช้ตรวจสอบสถานะการ export package สำหรับส่งไปยัง minio

In [17]:
is_success = dict(
  person_telephone = False,
  person = False,
  person_death = False,
  service_opd = False,
  service_opd_screening = False
)

## export to person_telephone

In [18]:
# person_telephone = set_package(service_opd_screening, ["cid"], ["telephone", "type", "ext"])
person_telephone = dict()
person_telephone["cid"] = service_opd_screening["cid"]
person_telephone["telephone"] = ""
person_telephone["type"] = ""
person_telephone["ext"] = ""

person_telephone = pd.DataFrame(person_telephone)
is_succuess = is_success["person_telephone"] = upload_minio(
    minio_config, person_telephone, f"{destination_path}/person_telephone.csv.gz", **api_config)

# display(service_opd_screening.__len__())
# display(person_telephone)

,cid,telephone,type,ext
0,3310401448357,,,
1,3310400125406,,,
2,1240600312930,,,
3,3310401337300,,,
4,3310400781826,,,
...,...,...,...,...
1054,<NA>,,,
1055,1319800851955,,,
1056,<NA>,,,
1057,1319800316120,,,


{'person_telephone': True,
 'person': False,
 'person_death': False,
 'service_opd': False,
 'service_opd_screening': False}

## export to person

In [19]:
# person = set_package(service_opd_screening, ["cid", "birhtdate","gender_code","gender_name","rh_blood_group_code","rh_blood_group_name","blood_group_code","blood_group_name","title_code","title_name","first_name","middle_name","last_name","nationality_code","nationality_name","marital_status_code","marital_status_name"], [])
# is_success["person_telephone"] = upload_minio(
#     minio_config, person_telephone, f"{destination_path}/person.csv.gz", **api_config)

person = dict()
person["cid"] = service_opd_screening["cid"]
person["birthdate"] = service_opd_screening["birthdate"]
person["gender_code"] = service_opd_screening["gender_stdcode"]
person["gender_name"] = service_opd_screening["gender_stdname_th"]
person["rh_blood_group_code"] = service_opd_screening["rh_blood_group_stdcode"]
person["rh_blood_group_name"] = service_opd_screening["rh_blood_group_stdname_th"]
person["blood_group_code"] = service_opd_screening["blood_group_stdcode"]
person["blood_group_name"] = service_opd_screening["blood_group_stdname_th"]
person["title_code"] = service_opd_screening["title_stdcode"]
person["title_name"] = service_opd_screening["title_stdname_th"]
person["first_name"] = service_opd_screening["first_name"]
person["middle_name"] = service_opd_screening["middle_name"]
person["last_name"] = service_opd_screening["last_name"]
person["nationality_code"] = service_opd_screening["nationality_stdcode"]
person["nationality_name"] = service_opd_screening["nationality_stdname_th"]
person["marital_status_code"] = service_opd_screening["marital_status_stdcode"]
person["marital_status_name"] = service_opd_screening["marital_status_stdname_th"]
person = pd.DataFrame(person)
is_success["person"] = upload_minio(
    minio_config, person, f"{destination_path}/person.csv.gz", **api_config)

# display(person.__len__())
# display(person)


## export to person_death 

## export to service_opd

## export to service_opd_screening

## export to service_opd_diagnosis


## export to service_opd_drugs


## export to service_appointment

## export to service_ipd

## service_ipd_diagnosis

## service_ipd_drugs

## summary success

In [20]:
is_success = dict(
    person_telephone=False,
    person=False,
    person_death=False,
    service_opd=False,
    service_opd_screening=False
)

if list(is_success.values()).count(False)>0:
  print("ERROR")
else:
  print("SUCCESS")
  

ERROR
